In [1]:
# Блок 1 — Чтение XML и получение ID
import xml.etree.ElementTree as ET

def parse_mod_ids(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    mods = []
    for workshop in root.findall("Workshop"):
        mod_id = workshop.get("id")
        name = workshop.get("name")
        if mod_id:
            mods.append((mod_id, name))
    return mods

# 🔧 Укажи путь к XML-файлу
xml_file = r"C:\Program Files (x86)\Steam\steamapps\common\Barotrauma\ModLists\Модное V8.xml"

# ✅ Тест: загрузка и вывод ID
mods = parse_mod_ids(xml_file)
print(f"Найдено {len(mods)} модов.")
for mod_id, name in mods[:5]:  # первые 5 для примера
    print(f"{mod_id} — {name}")

Найдено 128 модов.
2559634234 — Lua For Barotrauma
2795927223 — CsForBarotrauma
2701251094 — Performance Fix
3329396988 — NetworkTweaks
2807566753 — Dont Open Debug Console On Errors


In [2]:
# Блок 1 — Чтение XML и получение ID вторая 
import xml.etree.ElementTree as ET

# 🔧 Укажи путь к XML-файлу
xml_file = r"C:\Program Files (x86)\Steam\steamapps\common\Barotrauma\ModLists\Модное без Нейротравмы V5.xml"

# ✅ Тест: загрузка и вывод ID
mods1 = parse_mod_ids(xml_file)

mod_ids1 = set(mod_id for mod_id, _ in mods1)
mod_ids2 = set(mod_id for mod_id, _ in mods)

# Моды, которые есть в первом, но нет во втором
only_in_1 = mod_ids1 - mod_ids2
# Моды, которые есть во втором, но нет в первом
only_in_2 = mod_ids2 - mod_ids1

print("🔷 Моды только в первом списке:")
for mod_id in only_in_1:
    name = next(name for mid, name in mods1 if mid == mod_id)
    print(f"{mod_id} — {name}")

print("\n🔶 Моды только во втором списке:")
for mod_id in only_in_2:
    name = next(name for mid, name in mods2 if mid == mod_id)
    print(f"{mod_id} — {name}")


🔷 Моды только в первом списке:
3074045632 — Immersive Diving Gear
2811300302 — BetterFabricatorUI
2984452754 — DynamicEuropa - COL Compatibility Patch
3028141718 — Combat Overhaul Legacy
2946617451 — Unoculus (Subtitles n Real Silent Update)
2636444311 — EK_Berith_Mk-II
3404418999 — Ragdoll Cam
3045796581 — Enhanced Reactors
3409864551 — Hydroponics Patched with Cigarettes fixed rubber and Sulphur
3074073458 — Immersive Diving Gear - Real Sonar Compatibility Patch
2856758496 — Lua For Barotrauma (with xpath patch)

🔶 Моды только во втором списке:


NameError: name 'mods2' is not defined

In [3]:
import requests
from datetime import datetime

def fetch_single_mod(mod_id):
    url = "https://api.steampowered.com/ISteamRemoteStorage/GetPublishedFileDetails/v1/"
    data = {
        "itemcount": 1,
        "publishedfileids[0]": mod_id
    }
    response = requests.post(url, data=data)
    response.raise_for_status()
    result = response.json()["response"]["publishedfiledetails"][0]
    return result

def format_unix(timestamp):
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

# ✅ Тест на одном ID
test_mod_id = mods[0][0]
mod_data = fetch_single_mod(test_mod_id)

# 📋 Извлекаем поля
title = mod_data.get("title", "Без названия")
description = mod_data.get("description", "").strip()
created = format_unix(mod_data.get("time_created", 0))
updated = format_unix(mod_data.get("time_updated", 0))
subscriptions = mod_data.get("subscriptions", 0)
favorites = mod_data.get("favorited", 0)
lifetime_subs = mod_data.get("lifetime_subscriptions", 0)
lifetime_fav = mod_data.get("lifetime_favorited", 0)
views = mod_data.get("views", 0)
visibility = mod_data.get("visibility", 0)
banned = mod_data.get("banned", 0)
tags = [tag["tag"] for tag in mod_data.get("tags", [])]

# ✅ Форматированный вывод
print(f"=== {title} ===")
print(f"Создано: {created}")
print(f"Обновлено: {updated}")
print(f"Видимость: {visibility} | Забанен: {'Да' if banned else 'Нет'}")
print(f"Просмотры: {views}")
print(f"Подписки: {subscriptions} (всего: {lifetime_subs})")
print(f"Избранное: {favorites} (всего: {lifetime_fav})")
print(f"Теги: {', '.join(tags)}")
print("\nОписание:\n" + description[:1000] + ("..." if len(description) > 1000 else ""))


=== Lua For Barotrauma ===
Создано: 2021-07-28 16:10:35
Обновлено: 2025-04-29 19:33:49
Видимость: 0 | Забанен: Нет
Просмотры: 389897
Подписки: 489586 (всего: 554259)
Избранное: 16579 (всего: 17741)
Теги: Total conversion

Описание:
[h1]Updated for the Calm Before The Storm v1.8.8.1[/h1]
[h3]Discord: https://discord.gg/f9zvNNuxu9[/h3]
[h3]Consider supporting the project:[/h3]
[h3]https://patreon.com/Evil_Factory[/h3]
[h3]https://github.com/sponsors/evilfactory[/h3]

This is a Barotrauma modification that adds Lua modding, and [b]optional[/b] CSharp modding, this is not a direct replacement for xml, but works great in doing things that weren't possible in xml.

[hr]

[h2]Some Info[/h2]
Client-side mods run on your game client, whether you're playing singleplayer or joining a server. Server-side mods run on the server, which applies when you're hosting, meaning both client and server mods are active.

It's important to know that some mods require only server-side, some only client-side, a

In [4]:
import time

def fetch_all_mod_details(mods):
    url = "https://api.steampowered.com/ISteamRemoteStorage/GetPublishedFileDetails/v1/"
    all_data = {}
    chunk_size = 100
    for i in range(0, len(mods), chunk_size):
        chunk = mods[i:i+chunk_size]
        data = {"itemcount": len(chunk)}
        for idx, (mod_id, _) in enumerate(chunk):
            data[f"publishedfileids[{idx}]"] = mod_id
        response = requests.post(url, data=data)
        response.raise_for_status()
        results = response.json()["response"]["publishedfiledetails"]
        for original, mod in zip(chunk, results):
            mod_id, fallback_name = original
            all_data[mod_id] = mod
        time.sleep(1)  # чтобы не упереться в лимит Steam
    return all_data

# ✅ Получаем все данные
mod_data_dict = fetch_all_mod_details(mods)
print(f"Получено {len(mod_data_dict)} описаний модов.")



Получено 128 описаний модов.


In [5]:
from datetime import datetime

def format_unix(timestamp):
    return datetime.utcfromtimestamp(timestamp).strftime('%Y-%m-%d %H:%M:%S')

def save_full_mod_info(mod_data_dict, output_path):
    with open(output_path, "w", encoding="utf-8") as f:
        for mod_id, data in mod_data_dict.items():
            title = data.get("title", f"Мод {mod_id}")
            description = data.get("description", "").strip()
            created = format_unix(data.get("time_created", 0))
            updated = format_unix(data.get("time_updated", 0))
            subscriptions = data.get("subscriptions", 0)
            lifetime_subs = data.get("lifetime_subscriptions", 0)
            favorited = data.get("favorited", 0)
            lifetime_fav = data.get("lifetime_favorited", 0)
            views = data.get("views", 0)
            tags = [tag["tag"] for tag in data.get("tags", [])]

            f.write(f"=== {title} ===\n")
            f.write(f"ID: {mod_id}\n")
            f.write(f"Создано: {created}\n")
            f.write(f"Обновлено: {updated}\n")
            f.write(f"Просмотры: {views}\n")
            f.write(f"Подписки: {subscriptions} (всего: {lifetime_subs})\n")
            f.write(f"Избранное: {favorited} (всего: {lifetime_fav})\n")
            f.write(f"Теги: {', '.join(tags)}\n\n")
            f.write(description + "\n\n\n")

# ✅ Сохраняем в файл
output_file = "steam_mods_full_info.txt"
save_full_mod_info(mod_data_dict, output_file)
print(f"Сохранено в файл: {output_file}")


Сохранено в файл: steam_mods_full_info.txt
